# Logistic Regression

In [ ]:
def mapFeature(X1,X2):
    degree = 2;                     
    out = np.ones((X1.shape[0],1))

    for i in np.arange(1,degree+1): 
        for j in range(i+1):
            temp = X1**(i-j)*(X2**j)   
            out = np.hstack((out, temp.reshape(-1,1)))
    return out

In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize

## Load Data

In [ ]:
data = np.loadtxt("/content/MyDrive/MyDrive/Machine Learning Data/EX_2/ex2data1.txt",delimiter = ",")

X = data[:,0:2]
y = data[:,2]
m = X.shape[0]

# print(X,y,m)

## Plot Data

In [ ]:
def plot_data(X,y):
  pos = y == 1
  neg = y == 0
  
  plt.plot(X[pos, 0], X[pos, 1], 'k*', lw=2, ms=10)
  plt.plot(X[neg, 0], X[neg, 1], 'ko', mfc='y', ms=8, mec='k', mew=1)



In [ ]:
plot_data(X, y)

plt.xlabel('Exam 1 score')
plt.ylabel('Exam 2 score')
plt.legend(['Admitted', 'Not admitted'])


## Define Sigmoid Function

In [ ]:
def sigmoid(z):
  z = np.array(z)
  g = np.zeros(z.shape)

  g = 1/(1 + np.exp(-z))

  return g

In [ ]:
z = 0
g = sigmoid(z)

print('g(', z, ') = ', g)

In [ ]:
m , n = X.shape

X = np.concatenate([np.ones((m,1)),X],axis = 1)
# print(X)

## Cost Function and Gradient Descent

In [ ]:
def cost_func(theta,X,y):

  m = y.size
  J = 0
  grad = np.zeros(theta.shape)

  h = sigmoid(np.dot(X,theta))
  
  J = -(1/m) * np.sum(np.dot(y,(np.log(h))) + np.dot(1-y,(np.log(1-h))))

  grad = (1/m) * (h-y).dot(X)

  return J,grad


In [ ]:
initial_theta = np.zeros(n+1)

cost, grad = cost_func(initial_theta, X, y)

print('Cost at initial theta (zeros): {:.3f}'.format(cost))
print('Expected cost (approx): 0.693\n')

print('Gradient at initial theta (zeros):')
print('\t[{:.4f}, {:.4f}, {:.4f}]'.format(*grad))
print('Expected gradients (approx):\n\t[-0.1000, -12.0092, -11.2628]\n')

# Compute and display cost and gradient with non-zero theta
test_theta = np.array([-24, 0.2, 0.2])
cost, grad = cost_func(test_theta, X, y)

print('Cost at test theta: {:.3f}'.format(cost))
print('Expected cost (approx): 0.218\n')

print('Gradient at test theta:')
print('\t[{:.3f}, {:.3f}, {:.3f}]'.format(*grad))
print('Expected gradients (approx):\n\t[0.043, 2.566, 2.647]')

In [ ]:
options= {'maxiter': 400}

res = optimize.minimize(cost_func,
                        initial_theta,
                        (X, y),
                        jac=True,
                        method='TNC',
                        options=options)

cost = res.fun

theta = res.x

print('Cost at theta found by optimize.minimize: {:.3f}'.format(cost))
print('Expected cost (approx): 0.203\n');

print('theta:')
print('\t[{:.3f}, {:.3f}, {:.3f}]'.format(*theta))
print('Expected theta (approx):\n\t[-25.161, 0.206, 0.201]')

## Predictions

In [ ]:
def predict(theta,X):

  m = y.size
  
  p = np.zeros(m)

  p = np.round(sigmoid(np.dot(X,theta)))
  
  return p


In [ ]:
prob = sigmoid(np.dot([1, 45, 85], theta))
print('For a student with scores 45 and 85,'
      'we predict an admission probability of {:.3f}'.format(prob))
print('Expected value: 0.775 +/- 0.002\n')

# Compute accuracy on our training set
p = predict(theta, X)
print('Train Accuracy: {:.2f} %'.format(np.mean(p == y) * 100))
print('Expected accuracy (approx): 89.00 %')

# Logistic Regression with Regularization

In [ ]:
Data = np.loadtxt("/content/MyDrive/MyDrive/Machine Learning Data/EX_2/ex2data2.txt",delimiter = ",")

X = Data[:, 0:2]
y = Data[:, 2]
m = y.size

# print(X,y,m)

In [ ]:
plot_data(X,y)

plt.xlabel('Microchip Test 1')
plt.ylabel('Microchip Test 2')

plt.legend(['y = 1', 'y = 0'], loc='upper right')

In [ ]:
import utils
X = mapFeature(X[:, 0], X[:, 1])

In [ ]:
def regularization_cost_func(theta,X,y,lambda_):
  m = y.size
  J = 0
  grad = np.zeros(theta.shape)

  h = sigmoid(np.dot(X,theta))

  d = theta
  d[0] = 0

  J = (1/m) * np.sum(-np.dot(y,(np.log(h))) - np.dot(1-y,(np.log(1-h)))) + (lambda_/(2*m)) * np.sum(np.square(d))

  grad = (1 / m) * (h - y).dot(X) +  (lambda_ / m) * d

  return J,grad


In [ ]:
# Initialize fitting parameters
initial_theta = np.zeros(X.shape[1])

# Set regularization parameter lambda to 1
# DO NOT use `lambda` as a variable name in python
# because it is a python keyword
lambda_ = 1

# Compute and display initial cost and gradient for regularized logistic
# regression
cost, grad = regularization_cost_func(initial_theta, X, y, lambda_)

print('Cost at initial theta (zeros): {:.3f}'.format(cost))
print('Expected cost (approx)       : 0.693\n')

print('Gradient at initial theta (zeros) - first five values only:')
print('\t[{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}]'.format(*grad[:5]))
print('Expected gradients (approx) - first five values only:')
print('\t[0.0085, 0.0188, 0.0001, 0.0503, 0.0115]\n')


# Compute and display cost and gradient
# with all-ones theta and lambda = 10
test_theta = np.ones(X.shape[1])
cost, grad = regularization_cost_func(test_theta, X, y, 10)

print('------------------------------\n')
print('Cost at test theta    : {:.2f}'.format(cost))
print('Expected cost (approx): 3.16\n')

print('Gradient at initial theta (zeros) - first five values only:')
print('\t[{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}]'.format(*grad[:5]))
print('Expected gradients (approx) - first five values only:')
print('\t[0.3460, 0.1614, 0.1948, 0.2269, 0.0922]')